# Telegram Analysis

## Prepare environment

In [1]:
"""
Set flag to true, if you work in visual studio code (connected to custom docker)
Set flag to false, if you work in browser (jupyter notebook ui from custom docker)
"""
FLAG_LOCAL              = False
"""
Set flag to true, if you want to work an a spot check (quick run)
"""
FlAG_WORK_ON_SPOT_CHECK = False

In [2]:
# Import default libs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time

# Import url libs
from urllib.parse import urlparse
from collections import Counter

# Set graph widget (used by jupyter notebook)
#%matplotlib notebook   #interactive graphs
%matplotlib inline

In [3]:
# Install and import Graph Lib
import networkx as nx
! pip install pydot

In [4]:
# Install and import  JSON Lib
! pip install demjson
import demjson

In [5]:
# Install and import Natural Language Toolkit
! pip install nltk
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Install and import WordCloud
! pip install wordcloud
from wordcloud import WordCloud

In [7]:
# Show all columns (pandas hides columns by default)
pd.set_option('display.max_columns', None)

In [8]:
# Set env vars
if(FLAG_LOCAL == True):
    dir_var = "./work/notebooks/"
else:
    dir_var = "./"

dir_var_output = dir_var + "output/"

# Debug output
! echo "- Workdir -"
! ls -al $dir_var

! echo
! echo "- Outputdir -"
! ls -al $dir_var_output

- Workdir -
total 2964
drwxrwxr-x 5 jovyan  1000    4096 Jan  1 16:22 .
drwxrwxr-x 6 jovyan  1000    4096 Jan  1 10:38 ..
drwxrwxr-x 4 jovyan  1000    4096 Dec 20 10:21 data
-rw-rw-r-- 1 jovyan  1000    5295 Dec 20 10:51 inputFiles.csv
drwxr-xr-x 2 jovyan users    4096 Dec 20 10:26 .ipynb_checkpoints
drwxrwxr-x 2 jovyan  1000    4096 Jan  1 16:14 output
-rwxrwxr-x 1 jovyan  1000 3005911 Jan  1 16:22 Telegram.ipynb

- Outputdir -
total 3196
drwxrwxr-x 2 jovyan 1000   4096 Jan  1 16:14 .
drwxrwxr-x 5 jovyan 1000   4096 Jan  1 16:22 ..
-rw-rw-r-- 1 jovyan 1000      0 Dec 21 14:19 .gitkeep
-rw-rw-r-- 1 jovyan 1000  62484 Jan  1 16:19 meta-overview.png
-rw-rw-r-- 1 jovyan 1000  10889 Jan  1 16:19 meta-text-length-hist.png
-rw-rw-r-- 1 jovyan 1000 525852 Jan  1 16:20 social-graph.png
-rw-rw-r-- 1 jovyan 1000  36090 Jan  1 16:19 social-test-graph.png
-rw-rw-r-- 1 jovyan 1000 547417 Jan  1 16:21 wordcloud-global.png
-rw-rw-r-- 1 jovyan 1000 522478 Jan  1 16:20 wordcloud-pc-evaHerman.png
-rw-rw

### Global functions

In [9]:
"""
Check if text is json formatted

param   text        InputText
param   singleMode  Boolean (set to true, if text is part of a message)
"""
def gloCheckIsTextJsonFormatted(text, singleMode):
    textString = str(text)
    if      (singleMode == False and textString.startswith("[") == True and textString.endswith("]") == True):
        return True
    elif    (singleMode == True and textString.startswith("{") == True and textString.endswith("}") == True):
        return True
    else:
        return False

In [10]:
dictGloStopwatches = dict()

# Start timer (for reporting)
def gloStartStopwatch(key):
    print("[Stopwatch started >>" + str(key) + "<<]")
    dictGloStopwatches[key] = time.time()

# Stop timer (for reporting)
def gloStopStopwatch(key):
    endTime     = time.time()
    startTime   = dictGloStopwatches[key]
    print("[Stopwatch stopped >>" + str(key) + "<< (" + '{:5.3f}s'.format(endTime-startTime) + ")]")

In [11]:
# Generate unique chat name
def gloConvertToSafeChatName(chatName):
    chatName = chatName.encode('ascii', 'ignore')
    chatName = chatName.decode('ascii')
    return chatName[:30]  

## Process input jobs

In [12]:
# Read jobs from file
dfInputFiles = pd.read_csv(dir_var + "inputFiles.csv", sep=";")

In [13]:
# See description above (generate spot check)
if(FlAG_WORK_ON_SPOT_CHECK == True):
    dfFilter = pd.DataFrame()

    dfFilter = dfFilter.append(dfInputFiles[dfInputFiles.inputName.str.contains("Xavier Naidoo")])
    dfFilter = dfFilter.append(dfInputFiles[dfInputFiles.inputName.str.contains("Oliver Janich")])
    dfFilter = dfFilter.append(dfInputFiles[dfInputFiles.inputName.str.contains("Eva Herman")])
    dfFilter = dfFilter.append(dfInputFiles[dfInputFiles.inputName.str.contains("ATTILA HILDMANN")])

    dfInputFiles = dfFilter

### Overview input jobs

In [14]:
dfInputFiles

,inputName,inputPath,inputType,inputId,inputDesc,inputDownloadType
0,FREIHEITS-CHAT,DS-08-10-2020/ChatExport_2020-09-28-freiheitsChat,public_supergroup,9717909816,NaN,all
1,Oliver Janich oeffentlich,DS-08-10-2020/ChatExport_2020-09-25-janich,public_channel,9808932799,NaN,all
2,ATTILA HILDMANN OFFICIAL,DS-08-10-2020/ChatExport_2020-09-25-hildmann,public_channel,10034163583,NaN,all
3,Gerechtigkeit fuer das Vaterland,DS-08-10-2020/ChatExport_2020-09-26-gerechtigk...,public_channel,10069007089,NaN,all
4,Corona Virus Informationen,DS-08-10-2020/ChatExport_2020-09-26-cvirusinfo,public_channel,9917074801,NaN,all
5,Liberte,DS-08-10-2020/ChatExport_2020-09-26-liberte,public_channel,10068807626,NaN,all
6,Golden Age Network,DS-08-10-2020/ChatExport_2020-09-26-goldenage,public_channel,9862302472,NaN,all
7,Weltfrieden 2020,DS-08-10-2020/ChatExport_2020-09-26-weltfriede...,public_channel,9850980068,NaN,all
8,Verschwoerungen,DS-08-10-2020/ChatExport_2020-09-26-verschwoer...,public_channel,9942502804,NaN,all
9,Digital Research Army,DS-08-10-2020/ChatExport_2020-09-27-digitalRes...,public_channel,9846664545,NaN,all


### Transform data into DataFrmaes

In [15]:
# Convert to DataFrame Meta (Chat Meta)
def convertToDataFrameMeta(filePath):
    dF = pd.read_json(dir_var + "data/" + filePath + "/result.json", encoding='utf-8')
    return dF

# Convert to DataFrame Messages (Chat Messages)
def convertToDataFrameMessages(filePath, dictMeta):
    dF = pd.json_normalize(dictMeta[filePath].messages)
    return dF

In [16]:
"""
Get params from extractedTextData
See cell below (key)
"""
def getExtractedTextDataParam(key, extractedTextData):
    a,b,c,d,e,f,g = extractedTextData
    switcher = {
        0: a,
        1: b,
        2: c,
        3: d,
        4: e,
        5: f,
        6: g
    }
    return switcher.get(key)

In [17]:
# TODO: href, mention and hashtags in normal text?

"""
Extract text data (see cell above key)
See cell above (key)

param   procIsJsonFormatted Boolean (is text json formatted?)
param   text                String  (text from message) 

return
a   procText            Plain Text
b   processedURLs       Array of URLs in Text
c   processedHashtags   Array of Hashtags in Text
d   processedBolds      Array of Bold Items in Text
e   processedItalics    Array of Italic Items in Text
f   processedUnderlines Array of Underlined Items in Text
g   processedEmails     Array of E-Mails in Text
"""
def extractTextData(procIsJsonFormatted, text):
    
    # 3 returns in this function...
    
    processedURLs       = list()
    processedHashtags   = list()
    processedBolds      = list()
    processedItalics    = list()
    processedUnderlines = list()
    processedEmails     = list()
    
    if(procIsJsonFormatted != True):
        #Is not JSON formatted (return normal text)
        return (text, processedURLs, processedHashtags, processedBolds, processedItalics, processedUnderlines, processedEmails)
    else:
        #Is is JSON formatted (try to parse)
        try:
            returnList = []
            jsonList = demjson.decode(str(text), encoding='utf8')

            # Do for each item in list
            for lItem in jsonList:

                messageString = str(lItem)

                isJsonSubString = gloCheckIsTextJsonFormatted(messageString, singleMode = True)

                if(isJsonSubString):
                    # Is Json Sub String
                    subJsonString = demjson.decode(str(messageString), encoding='utf8')
                    subJsonType = subJsonString["type"]

                    if(subJsonType == "bold"):
                        #text included
                        processedBolds.append(subJsonString["text"])
                        returnList.append(subJsonString["text"])
                        
                    elif(subJsonType == "italic"):
                        #text included
                        processedItalics.append(subJsonString["text"])
                        returnList.append(subJsonString["text"])
                        
                    elif(subJsonType == "underline"):
                        #text included
                        processedUnderlines.append(subJsonString["text"])
                        returnList.append(subJsonString["text"])
                    
                    elif(subJsonType == "email"):
                        #text included
                        processedEmails.append(subJsonString["text"])
                        
                    elif(subJsonType == "text_link"):
                        #text and href included
                        processedURLs.append(subJsonString["href"])
                        returnList.append(subJsonString["text"])
                        
                    elif(subJsonType == "link"):
                        #text included
                        processedURLs.append(subJsonString["text"])
                        
                    elif(subJsonType == "hashtag"):
                        #text included
                        processedHashtags.append(subJsonString["text"])
                        returnList.append(subJsonString["text"])
                        
                    elif(subJsonType == "mention"):
                        #text included
                        returnList.append(subJsonString["text"])
                        
                    elif(subJsonType == "mention_name"):
                        #text and user_id included
                        returnList.append(subJsonString["text"])
                        
                    elif(subJsonType == "bot_command"):
                        #text included
                        returnList = returnList 
                        
                    elif(subJsonType == "code"):
                        #text included
                        returnList = returnList
                        
                    elif(subJsonType == "phone"):
                        #text included
                        returnList = returnList
                        
                    elif(subJsonType == "strikethrough"):
                        #text included
                        returnList.append(subJsonString["text"])
                        
                    elif(subJsonType == "pre"):
                        #text and language included
                        returnList.append(subJsonString["text"])
                        
                    elif(subJsonType == "bank_card"):
                        #text included
                        returnList = returnList
                        
                    else:
                        print("- Error: Unkown json type >>" + str(subJsonType) + "<< (ignore) >>" + str(text) + "<<")

                else:
                    # Is no json formatted sub string (append text)
                    returnList.append(messageString)

            return (''.join(returnList), processedURLs, processedHashtags, processedBolds, processedItalics, processedUnderlines, processedEmails)
        
        except:
            # Parser error (set inputText to returnText)
            print("- Warn: Json parser error (set inputText to returnText) >>" + str(text) + "<<")
            return (text, processedURLs, processedHashtags, processedBolds, processedItalics, processedUnderlines, processedEmails)

In [18]:
def evalIsValidText(procTDTextLength):
    if(procTDTextLength > 0):
        return True
    else:
        return False

In [19]:
def evalContainsSomething(att):
    if(str(att) == "nan"):
        return False
    else:
        return True

In [20]:
gloStartStopwatch("Extract Text Data")

# Add Key = filePath / Value = DataFrame (Chat Meta)
dictMeta = {}
for fP in dfInputFiles.inputPath:
    dictMeta[fP] = convertToDataFrameMeta(fP)

# Add Key = filePath / Value = DataFrame (Chat Message)
dictMessages = {}
for fP in dfInputFiles.inputPath:

    gloStartStopwatch("TD-Extract " + fP)
    dfMessages                          = convertToDataFrameMessages(fP, dictMeta)
    
    # Get chat attributes and check if message is json formatted
    dfMessages["procChatFilePath"]      = fP
    dfMessages["procChatType"]          = dictMeta[fP].type.iloc[0]
    dfMessages["procIsJsonFormatted"]   = dfMessages["text"].apply(gloCheckIsTextJsonFormatted, singleMode = False)
    
    # Extract Text Data
    dfMessages["tmpExtractedTD"]        = dfMessages.apply(lambda x: extractTextData(x.procIsJsonFormatted, x.text), axis=1)

    # Extract Text Data (params)
    dfMessages["procTDText"]            = dfMessages.apply(lambda x: getExtractedTextDataParam(0, x.tmpExtractedTD), axis=1)
    dfMessages["procTDURLs"]            = dfMessages.apply(lambda x: getExtractedTextDataParam(1, x.tmpExtractedTD), axis=1)
    dfMessages["procTDHashtags"]        = dfMessages.apply(lambda x: getExtractedTextDataParam(2, x.tmpExtractedTD), axis=1)
    dfMessages["procTDBolds"]           = dfMessages.apply(lambda x: getExtractedTextDataParam(3, x.tmpExtractedTD), axis=1)
    dfMessages["procTDItalics"]         = dfMessages.apply(lambda x: getExtractedTextDataParam(4, x.tmpExtractedTD), axis=1)
    dfMessages["procTDUnderlines"]      = dfMessages.apply(lambda x: getExtractedTextDataParam(5, x.tmpExtractedTD), axis=1)
    dfMessages["procTDEmails"]          = dfMessages.apply(lambda x: getExtractedTextDataParam(6, x.tmpExtractedTD), axis=1)

    # Calc text size
    dfMessages["procTDTextLength"]      = dfMessages["procTDText"].str.len()

    # Add columns (if not exists)
    if "photo" not in dfMessages:
        print("- Warn: Add column >>photo<<")
        dfMessages["photo"] = np.nan

    if "file" not in dfMessages:
        print("- Warn: Add column >>file<<")
        dfMessages["file"] = np.nan

    if "edited" not in dfMessages:
        print("- Warn: Add column >>edited<<")
        dfMessages["edited"] = np.nan

    if "forwarded_from" not in dfMessages:
        print("- Warn: Add column >>forwarded_from<<")
        dfMessages["forwarded_from"] = np.nan

    # Evaluate attributes
    dfMessages["procEvalIsValidText"]   = dfMessages.procTDTextLength.apply(evalIsValidText)
    dfMessages["procEvalContainsPhoto"] = dfMessages.photo.apply(evalContainsSomething)
    dfMessages["procEvalContainsFile"]  = dfMessages.file.apply(evalContainsSomething) 
    dfMessages["procEvalIsEdited"]      = dfMessages.edited.apply(evalContainsSomething)
    dfMessages["procEvalIsForwarded"]   = dfMessages.forwarded_from.apply(evalContainsSomething)
    # TODO: Add more attributes (e.g. procEvalContainBoldItems)
    
    dictMessages[fP] = dfMessages
    gloStopStopwatch("TD-Extract " + fP)

    
# All Messages to DataFrame
gloStartStopwatch("Generate global DataFrame")
dfAllDataMessages = pd.DataFrame()
for fP in dfInputFiles.inputPath:
    dfMessages        = dictMessages[fP].copy()
    dfAllDataMessages = dfAllDataMessages.append(dfMessages)
gloStopStopwatch("Generate global DataFrame")

gloStopStopwatch("Extract Text Data")

[Stopwatch started >>Extract Text Data<<]
[Stopwatch started >>TD-Extract DS-08-10-2020/ChatExport_2020-09-28-freiheitsChat<<]
- Warn: Json parser error (set inputText to returnText) >>[Nachweis hier einfügen]<<
[Stopwatch stopped >>TD-Extract DS-08-10-2020/ChatExport_2020-09-28-freiheitsChat<< (259.688s)]
[Stopwatch started >>TD-Extract DS-08-10-2020/ChatExport_2020-09-25-janich<<]
[Stopwatch stopped >>TD-Extract DS-08-10-2020/ChatExport_2020-09-25-janich<< (85.783s)]
[Stopwatch started >>TD-Extract DS-08-10-2020/ChatExport_2020-09-25-hildmann<<]
[Stopwatch stopped >>TD-Extract DS-08-10-2020/ChatExport_2020-09-25-hildmann<< (29.729s)]
[Stopwatch started >>TD-Extract DS-08-10-2020/ChatExport_2020-09-26-gerechtigkeitVaterland<<]
[Stopwatch stopped >>TD-Extract DS-08-10-2020/ChatExport_2020-09-26-gerechtigkeitVaterland<< (1.396s)]
[Stopwatch started >>TD-Extract DS-08-10-2020/ChatExport_2020-09-26-cvirusinfo<<]
[Stopwatch stopped >>TD-Extract DS-08-10-2020/ChatExport_2020-09-26-cvirusinf

[Stopwatch stopped >>TD-Extract DS-22-10-2020/ChatExport_2020-10-18-freiheitDerGedanken<< (9.163s)]
[Stopwatch started >>TD-Extract DS-22-10-2020/ChatExport_2020-10-18-querdenken711<<]
[Stopwatch stopped >>TD-Extract DS-22-10-2020/ChatExport_2020-10-18-querdenken711<< (1.807s)]
[Stopwatch started >>TD-Extract DS-22-10-2020/ChatExport_2020-10-18-holistischeGesundheitHeilung<<]
[Stopwatch stopped >>TD-Extract DS-22-10-2020/ChatExport_2020-10-18-holistischeGesundheitHeilung<< (9.882s)]
[Stopwatch started >>TD-Extract DS-22-10-2020/ChatExport_2020-10-20-faktenFriedenFreiheit<<]
[Stopwatch stopped >>TD-Extract DS-22-10-2020/ChatExport_2020-10-20-faktenFriedenFreiheit<< (12.646s)]
[Stopwatch started >>TD-Extract DS-22-10-2020/ChatExport_2020-10-21-babylonDecoded<<]
- Warn: Add column >>file<<
- Warn: Add column >>forwarded_from<<
[Stopwatch stopped >>TD-Extract DS-22-10-2020/ChatExport_2020-10-21-babylonDecoded<< (0.513s)]
[Stopwatch started >>TD-Extract DS-22-10-2020/ChatExport_2020-10-21-z

## Meta-Queries

### Type of channels (and only included in)

In [21]:
dfInputFiles.inputType.value_counts()

public_channel        38
public_supergroup      8
private_supergroup     1
Name: inputType, dtype: int64

In [22]:
#dfAllDataMessages.columns.difference(dfAllDataMessages[dfAllDataMessages.procChatType == "public_channel"].dropna(how='all', axis=1).columns)

In [23]:
#dfAllDataMessages.columns.difference(dfAllDataMessages[dfAllDataMessages.procChatType == "public_supergroup"].dropna(how='all', axis=1).columns)

In [24]:
# No enough data
#dfAllDataMessages.columns.difference(dfAllDataMessages[dfAllDataMessages.procChatType == "private_supergroup"].dropna(how='all', axis=1).columns)

### Define queries

In [25]:
def queryChatId(filePath):
    dfMeta = dictMeta[filePath].copy()
    return str(dfMeta["id"].iloc[0])

In [26]:
def queryChatName(filePath):
    dfMeta      = dictMeta[filePath].copy()
    chatName    = str(dfMeta["name"].iloc[0])
    chatName    = gloConvertToSafeChatName(chatName)
    return chatName

In [27]:
def queryChatType(filePath):
    dfMeta = dictMeta[filePath].copy()
    return str(dfMeta["type"].iloc[0])

In [28]:
def queryNumberOfMessages(filePath):
    dfMessages = dictMessages[filePath].copy()
    return len(dfMessages.index)

In [29]:
def queryNumberOfFormattedTextMessages(filePath):
    dfMessages = dictMessages[filePath].copy()
    dfMessages = dfMessages[dfMessages.procIsJsonFormatted == True]
    return len(dfMessages.index)

In [30]:
def queryNumberOfValidTextMessages(filePath):
    dfMessages = dictMessages[filePath].copy()
    dfMessages = dfMessages[dfMessages.procEvalIsValidText == True]
    return len(dfMessages.index)

In [31]:
def queryNumberOfPhotos(filePath):
    dfMessages = dictMessages[filePath].copy()
    dfMessages = dfMessages[dfMessages.procEvalContainsPhoto == True]
    return len(dfMessages.index)

In [32]:
def queryNumberOfFiles(filePath):
    dfMessages = dictMessages[filePath].copy()
    dfMessages = dfMessages[dfMessages.procEvalContainsFile == True]
    return len(dfMessages.index)

In [33]:
def queryNumberOfEditedMessages(filePath):
    dfMessages = dictMessages[filePath].copy()
    dfMessages = dfMessages[dfMessages.procEvalIsEdited == True]
    return len(dfMessages.index)

In [34]:
def queryNumberOfForwardedMessages(filePath):
    dfMessages = dictMessages[filePath].copy()
    dfMessages = dfMessages[dfMessages.procEvalIsForwarded == True]
    return len(dfMessages.index)

### Execute queries

In [ ]:
dfQueryMeta = pd.DataFrame(dfInputFiles.inputPath)

dfQueryMeta["qryChatId"]                        = dfQueryMeta.inputPath.apply(queryChatId)
dfQueryMeta["qryChatName"]                      = dfQueryMeta.inputPath.apply(queryChatName)
dfQueryMeta["qryChatType"]                      = dfQueryMeta.inputPath.apply(queryChatType)
dfQueryMeta["qryNumberOfMessages"]              = dfQueryMeta.inputPath.apply(queryNumberOfMessages)
dfQueryMeta["qryNumberOfFormattedTextMessages"] = dfQueryMeta.inputPath.apply(queryNumberOfFormattedTextMessages)
dfQueryMeta["qryNumberOfValidTextMessages"]     = dfQueryMeta.inputPath.apply(queryNumberOfValidTextMessages)
dfQueryMeta["qryNumberOfPhotos"]                = dfQueryMeta.inputPath.apply(queryNumberOfPhotos)
dfQueryMeta["qryNumberOfFiles"]                 = dfQueryMeta.inputPath.apply(queryNumberOfFiles)
dfQueryMeta["qryNumberOfEditedMessages"]        = dfQueryMeta.inputPath.apply(queryNumberOfEditedMessages)
dfQueryMeta["qryNumberOfForwardedMessages"]     = dfQueryMeta.inputPath.apply(queryNumberOfForwardedMessages)

dfQueryMeta.sort_values(by="qryNumberOfMessages", ascending=False)

### Plot meta queries

In [ ]:
# Auto label query plot
def autolabelAx(rects, ax):
    """
    Attach a text label above each bar in *rects*, displaying its height.
    Copied from https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/barchart.html (22.12.2020)
    """
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

In [ ]:
def queryMetaPlotter():
    # Init data
    dataLabels                          = list()
    dataNumberOfMesssages               = list()
    dataNumberOfFormattedTextMessages   = list()
    dataNumberOfValidTextMessages       = list()
    dataNumberOfEditedMessages          = list()
    dataNumberOfPhotos                  = list()
    dataNumberOfFiles                   = list()
    dataNumberOfForwardedMessages       = list()

    # Iterate over Meta DataFrame
    for index, row in dfQueryMeta.sort_values(by="qryNumberOfMessages", ascending=False).iterrows():

        # Get attributes
        dataLabels                          .append(row.qryChatName)
        dataNumberOfMesssages               .append(row.qryNumberOfMessages)
        dataNumberOfFormattedTextMessages   .append(row.qryNumberOfFormattedTextMessages)
        dataNumberOfValidTextMessages       .append(row.qryNumberOfValidTextMessages)
        dataNumberOfEditedMessages          .append(row.qryNumberOfEditedMessages)
        dataNumberOfPhotos                  .append(row.qryNumberOfPhotos)
        dataNumberOfFiles                   .append(row.qryNumberOfFiles)
        dataNumberOfForwardedMessages       .append(row.qryNumberOfForwardedMessages)

    # Convert list to array
    dataLabels                          = np.array(dataLabels)
    dataNumberOfMesssages               = np.array(dataNumberOfMesssages)
    dataNumberOfFormattedTextMessages   = np.array(dataNumberOfFormattedTextMessages)
    dataNumberOfValidTextMessages       = np.array(dataNumberOfValidTextMessages)
    dataNumberOfEditedMessages          = np.array(dataNumberOfEditedMessages)
    dataNumberOfPhotos                  = np.array(dataNumberOfPhotos)
    dataNumberOfFiles                   = np.array(dataNumberOfFiles)
    dataNumberOfForwardedMessages       = np.array(dataNumberOfForwardedMessages)

    # Draw
    fig, ax = plt.subplots()
    x = np.arange(len(dataLabels))

    barWidth = 0.1

    fig.set_figwidth(64)
    fig.set_figheight(16)

    r1 = x
    r2 = [x + barWidth for x in r1]
    r3 = [x + barWidth for x in r2]
    r4 = [x + barWidth for x in r3]
    r5 = [x + barWidth for x in r4]
    r6 = [x + barWidth for x in r5]
    r7 = [x + barWidth for x in r6]

    rects1 = ax.bar(r1, dataNumberOfMesssages, barWidth, label='Messages')
    rects2 = ax.bar(r2, dataNumberOfFormattedTextMessages, barWidth, label='Formatted Messsages')
    rects3 = ax.bar(r3, dataNumberOfValidTextMessages, barWidth, label='Valid Messages')
    rects4 = ax.bar(r4, dataNumberOfEditedMessages, barWidth, label='Edited Messages')
    rects5 = ax.bar(r5, dataNumberOfPhotos, barWidth, label='with Photo')
    rects6 = ax.bar(r6, dataNumberOfFiles, barWidth, label='with File')
    rects7 = ax.bar(r7, dataNumberOfForwardedMessages, barWidth, label='Forwarded Messages')

    ax.set_ylabel("Count")
    ax.set_title("Meta Overview")
    ax.set_xticks(x)
    ax.set_xticklabels(dataLabels)
    ax.legend()

    rects = [rects1, rects2, rects3, rects4, rects5, rects6, rects7]

    for rect in rects:
        autolabelAx(rect, ax)

    fig.tight_layout()

    plt.xticks(rotation=30)
    plt.savefig(dir_var_output + "meta-overview.png")
    plt.show()

In [ ]:
queryMetaPlotter()

### Get text-length

In [ ]:
def removeTextLengthOutliersFromDataFrame(df, interval, maxTextLength):
    df = df.copy()
    df = df[df.procTDTextLength < maxTextLength]
    # https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame
    # keep only the ones that are within <interval> to -<interval> standard deviations in the column 'Data'.
    return df[np.abs(df.procTDTextLength-df.procTDTextLength.mean()) <= (interval*df.procTDTextLength.std())]

In [ ]:
def textLengthHistPlotter():
    dfMessages = dfAllDataMessages.copy()
    print("Number of all messages:\t\t\t\t\t\t" + str(len(dfMessages.index)))

    dfMessages = dfMessages[dfMessages.procEvalIsValidText == True]
    print("Number of valid text messages:\t\t\t\t\t" + str(len(dfMessages.index)))

    dfMessages = removeTextLengthOutliersFromDataFrame(
        dfMessages,
        interval = 3,         #default is 3
        maxTextLength = 750
        )
    print("Number of valid text messages (after outliers filtering):\t" + str(len(dfMessages.index)))

    print()
    print("Text Length Hist (after outliers filtering)")
    plt.figure(figsize=(16,9))
    _ = dfMessages.procTDTextLength.hist(bins=10)

    plt.savefig(dir_var_output + "meta-text-length-hist.png")

In [ ]:
textLengthHistPlotter()

## Extract Important Hashtags

In [ ]:
# TODO: Extract hashtags in non formatted text

def extractImportantHashtags(filePath):
    dfMessages = dictMessages[filePath].copy()
    dfMessages = dfMessages[dfMessages.astype(str)["procTDHashtags"] != "[]"]

    hashTagList = list()
    for index, row in dfMessages.iterrows():
        for hashtagItem in row["procTDHashtags"]:
            hashTagList.append(hashtagItem)

    return Counter(hashTagList).most_common(20)

In [ ]:
extractImportantHashtags("DS-08-10-2020/ChatExport_2020-09-25-janich")

In [ ]:
extractImportantHashtags("DS-08-10-2020/ChatExport_2020-09-27-evaherman")

In [ ]:
extractImportantHashtags("DS-08-10-2020/ChatExport_2020-09-25-hildmann")

In [ ]:
extractImportantHashtags("DS-08-10-2020/ChatExport_2020-09-25-xavier")

## Extract Social Graph

In [ ]:
# TODO: Bug: No Hostname detected if string startsWith ! "http" in urlparse
# TODO: Feature: Url in non formatted text?
# TODO: Feature: Add other attributes (e.g. hashtags)
# TODO: Feature: Cache-Layer: Check if both set (from and from_id, actor, ...)
# TODO: Check: Duplicates refs from text

# TODO: Doc: Text ref important for finding groups
# TODO: Doc: forwared_from important for graphs

# Return Counter forwarded_from
def extractSocialGraph(filePath, debugPrint):
    dfMessages = dictMessages[filePath].copy()

    hostList = list()
    urList   = list()
    refList = list()
    for index, row in dfMessages.iterrows():
        if(str(row["procTDURLs"]) != "[]"):
            for urlItem in row["procTDURLs"]:
                urlData = urlparse(str(urlItem))

                completeHostname = urlData.hostname
                completeUrl      = urlData.geturl()

                hostList.append(str(completeHostname))
                urList.append(str(completeUrl))

                if "t.me" in str(completeHostname):
                    refList.append(str(completeUrl))
            
    forwardedFromList = list()
    if("forwarded_from" in dfMessages.columns):
        for index, row in dfMessages.iterrows():        
            forwardedFromList.append(str(row["forwarded_from"]))
            
    actorList = list()
    if("actor" in dfMessages.columns):
        for index, row in dfMessages.iterrows():
            actorList.append(str(row["actor"]))
    
    memberList = list()
    if("members" in dfMessages.columns):
        for index, row in dfMessages.iterrows():
            if(str(row["members"]) != "nan"):
                for memberItem in row["members"]:
                    memberList.append(str(memberItem))
                    
    fromList = list()
    if("from" in dfMessages.columns):
        for index, row in dfMessages.iterrows():
            fromList.append(str(row["from"]))
            
    savedFromList = list()
    if("saved_from" in dfMessages.columns):
        for index, row in dfMessages.iterrows():
            savedFromList.append(str(row["saved_from"]))
        
    if(debugPrint):
        print("- Top 20 Hosts -")
        print ("\n".join(map(str, Counter(hostList).most_common(20))))
        print()
        print("- Top 20 URLs -")
        print ("\n".join(map(str, Counter(urList).most_common(20))))
        print()
        print("- Top 20 Refs from text -")
        print ("\n".join(map(str, Counter(refList).most_common(20))))
        print()
        print("- Top 20 Refs (forwarded_from) -")
        print ("\n".join(map(str, Counter(forwardedFromList).most_common(20))))
        print()
        print("- Top 20 Refs (actor) -")
        print ("\n".join(map(str, Counter(actorList).most_common(20))))
        print()
        print("- Top 20 Refs (members) -")
        print ("\n".join(map(str, Counter(memberList).most_common(20))))
        print()
        print("- Top 20 Refs (from) -")
        print ("\n".join(map(str, Counter(fromList).most_common(20))))
        print()
        print("- Top 20 Refs (saved_from) -")
        print ("\n".join(map(str, Counter(savedFromList).most_common(20))))
        print()
    
    return Counter(forwardedFromList)

In [ ]:
_ = extractSocialGraph("DS-08-10-2020/ChatExport_2020-09-25-janich", debugPrint = True)

In [ ]:
_ = extractSocialGraph("DS-08-10-2020/ChatExport_2020-09-27-evaherman", debugPrint = True)

In [ ]:
_ = extractSocialGraph("DS-08-10-2020/ChatExport_2020-09-25-hildmann", debugPrint = True)

In [ ]:
_ = extractSocialGraph("DS-08-10-2020/ChatExport_2020-09-25-xavier", debugPrint = True)

### Plot Social Graph

In [ ]:
# TODO: Refactor

# Only add node size greater than
def addSocialGraphNodeSize(label, newSize, targetDict):
    
    if(label in targetDict):
        oldSize = targetDict[label]
        if(newSize > oldSize):
            targetDict[label] = newSize
    else:
        targetDict[label] = newSize

In [ ]:
# TODO: Nan?
# TODO: https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout
# TODO: (try different layouts e.g. circular_layout)
# TODO: Dont add empty group names (no ref!)    
# TODO: Check distances

"""
Generate social graph

param   configTopNInfluencer    e.g. For top 10 = 10
param   configMinRefs           e.g. 1 must have > 1 % forwarded messages
"""
def generateSocialGraph(configTopNInfluencer, configMinRefs):
    
    dictSocialNodeSizes = dict()
    listExactValues     = list()
    
    gloStartStopwatch("Social Graph")
    
    #Use Graph instead of DiGraph (Converted by draw)
    G_weighted = nx.DiGraph()
    
    print("- Add edges")
    for fP in dfInputFiles.inputPath:
        
        chatName                        = queryChatName(fP)
        chatNumberOfMessages            = queryNumberOfMessages(fP)
        chatNumberOfForwardeMessages    = queryNumberOfForwardedMessages(fP)

        gloStartStopwatch("SG-Extract " + chatName + "(" + str(chatNumberOfMessages) + " messages)")
        
        # Add Correct Node Size (data downloaded)
        addSocialGraphNodeSize(chatName, chatNumberOfMessages, dictSocialNodeSizes)
        listExactValues.append(chatName)

        # Get Graph Data
        socialGraphData = extractSocialGraph(fP, debugPrint=False)
        socialGraphData = socialGraphData.most_common(configTopNInfluencer)
        
        for oChatName, oChatRefs in socialGraphData:
            
            oChatName    = gloConvertToSafeChatName(str(oChatName))
            oChatRefs    = oChatRefs

            if(oChatName != "nan"):
        
                per = (oChatRefs/chatNumberOfForwardeMessages) * 100

                if(per > configMinRefs):
                
                    # Add Incorrect Node Size (no data downloaded)
                    addSocialGraphNodeSize(oChatName, oChatRefs, dictSocialNodeSizes)

                    wei = 100 - per

                    # Add Edge
                    G_weighted.add_edge(
                        chatName,
                        oChatName,
                        weight=wei,
                        tLabel = str(round(per, 3)) + "% (" + str(oChatRefs) + "/" + str(chatNumberOfForwardeMessages) + "≙" + str(round(wei, 3)) + ")"
                        #tLabel = str(round(per, 3)) + "%"
                    )

        gloStopStopwatch("SG-Extract " + chatName + "(" + str(chatNumberOfMessages) + " messages)")
        
    print("- Add different node sizes")
    for aNode in dictSocialNodeSizes:
        
        nodeName = str(aNode)
        nodeSize = dictSocialNodeSizes[aNode]

        tValueColor = "#ff8000"

        if(nodeName in listExactValues):
            tValueColor = "#0080ff"
        
        G_weighted.add_node(
            nodeName,
            weight=nodeSize,
            tLabel = str(nodeName) + " [" + str(nodeSize) + "]",
            tColor=tValueColor
        )
        
    gloStopStopwatch("Social Graph")
        
    return G_weighted

In [ ]:
"""
Draw social media plot

param   G                           graph
param   configFactorEdge            e.g. 100 => weight / 100
param   configFactorNode            e.g. 10  => weight / 10
param   configArrowSize             e.g. 5
param   configPlotWidth             e.g. 16
param   configPlotHeight            e.g. 9
param   outputFilename              e.g. test.png
"""
def drawSocialPlot(G, configFactorEdge, configFactorNode, configArrowSize, configPlotWidth, configPlotHeight, outputFilename):
    
    gloStartStopwatch("Social Graph Plot")
    
    plt.figure(figsize=(configPlotWidth,configPlotHeight))
        
    #pos = nx.nx_pydot.graphviz_layout(G)
    #pos = nx.spring_layout(G.to_undirected(), k = 0.15, iterations=200)
    pos = nx.kamada_kawai_layout(G.to_undirected())
    
    # Clean edges
    edges       = nx.get_edge_attributes(G, "weight")
    edgesTLabel = nx.get_edge_attributes(G, "tLabel")

    clean_edges         = dict()
    clean_edges_labels  = dict()
    
    for key in edges:
        
        #Set line weight
        clean_edges[key]        = (100 - edges[key]) / configFactorEdge

        #set label
        clean_edges_labels[key] = edgesTLabel[key]
    
    # Clean nodes
    nodes       = nx.get_node_attributes(G,'weight')
    nodesTLabel = nx.get_node_attributes(G,'tLabel')
    nodesTColor = nx.get_node_attributes(G,'tColor')

    clean_nodes         = dict()
    clean_nodes_labels  = dict()
    clean_nodes_color   = dict()
    
    for key in nodes:
        
        #Set Node weight        
        clean_nodes[key]        = nodes[key] / configFactorNode

        #Set Node layout
        clean_nodes_labels[key] = nodesTLabel[key]
        clean_nodes_color[key]  = nodesTColor[key]
    
    # Revert DiGraph (arrows direction)
    G_rev = nx.DiGraph.reverse(G)    

    nx.draw(G_rev,
        pos,
        with_labels=True,
        width=list(clean_edges.values()),
        node_size=list(clean_nodes.values()),
        labels=clean_nodes_labels,
        node_color=list(clean_nodes_color.values()),
        arrowsize=configArrowSize,
        arrowstyle="wedge" #TODO: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.patches.ArrowStyle.html
        #connectionstyle="arc3, rad = 0.1"
    )
        
    _ = nx.draw_networkx_edge_labels(G_rev, pos, edge_labels=clean_edges_labels)

    plt.savefig(dir_var_output + outputFilename)
    plt.show()
    
    gloStopStopwatch("Social Graph Plot")

#### Test Graph

In [ ]:
# Generates Test Graph
def generateTestGraph():

    #Use Graph instead of DiGraph
    G_weighted = nx.DiGraph()

    G_weighted.add_edge("N1", "N2", weight=100-30,  tLabel = "(≙" + str(100-30) + ")")
    G_weighted.add_edge("N1", "N3", weight=100-10,  tLabel = "(≙" + str(100-10) + ")")
    G_weighted.add_edge("N1", "N4", weight=100-60,  tLabel = "(≙" + str(100-60) + ")")

    G_weighted.add_edge("N4", "N5", weight=100-80,  tLabel = "(≙" + str(100-80) + ")")
    G_weighted.add_edge("N4", "N6", weight=100-10,  tLabel = "(≙" + str(100-10) + ")")

    G_weighted.add_edge("N4", "N7", weight=100-30,   tLabel = "(≙" + str(100-30) + ")")
    G_weighted.add_edge("N7", "N4", weight=100-70,   tLabel = "(≙" + str(100-70) + ")")

    G_weighted.add_node("N1", weight=500.0, tLabel = "N1-T", tColor="red")
    G_weighted.add_node("N2", weight=500.0, tLabel = "N2-T", tColor="blue")
    G_weighted.add_node("N3", weight=500.0, tLabel = "N3-T", tColor="blue")
    G_weighted.add_node("N4", weight=500.0, tLabel = "N4-T", tColor="red")
    G_weighted.add_node("N5", weight=500.0, tLabel = "N5-T", tColor="red")
    G_weighted.add_node("N6", weight=500.0, tLabel = "N6-T", tColor="red")
    G_weighted.add_node("N7", weight=500.0, tLabel = "N7-T", tColor="blue")

    return G_weighted

generatedTestGraph = generateTestGraph()

drawSocialPlot(
    generatedTestGraph,
    configFactorEdge = 10,
    configFactorNode = 1,
    configArrowSize = 15,
    configPlotWidth = 16,
    configPlotHeight = 9,
    outputFilename = "social-test-graph.png"
)

#### Real Graph

In [ ]:
generatedSocialGraph = generateSocialGraph(
    configTopNInfluencer = 50,  # TODO: Refactor
    configMinRefs = 1,          # TODO: Refactor
)

In [ ]:
# Graph
drawSocialPlot(
    generatedSocialGraph,
    configFactorEdge = 100,
    configFactorNode = 1,
    configArrowSize = 15,
    configPlotWidth = 32,
    configPlotHeight = 32,
    outputFilename = "social-graph.png"
)

## Word Clouds

In [ ]:
# TODO: Difference capital letters?
# TODO: Context?
# TODO: Improve stop words
# TODO: Only on valid text?

"""
WordCloud

param   filePath    String (set to "global" if you want to process all messages)
param   label       filename in outputdir
param   filterList  Exclude list
"""

def plotWordCloud(filePath, label, filterList):
    
    gloStartStopwatch("Word Cloud")

    if(filePath != "global"):
        dfMessages = dictMessages[filePath].copy()
    else:
        dfMessages = dfAllDataMessages.copy()
    
    print("- Start transform text to global text string")
    textList = []
    for index, row in dfMessages.iterrows():
        textList.append(" " + row["procTDText"])
        
    textString = ''.join(textList)
    
    germanStopWordsList = nltk.corpus.stopwords.words('german')
    germanStopWordsList.append("http")
    germanStopWordsList.append("https")
    germanStopWordsList.append("ja")
    germanStopWordsList.append("nein")
    germanStopWordsList.append("mehr")
    germanStopWordsList.append("mal")
    germanStopWordsList.append("schon")
    germanStopWordsList.append("immer")
    germanStopWordsList.append("wurde")
    germanStopWordsList.append("wurden")
    germanStopWordsList.append("sei")
    germanStopWordsList.append("sein")
    germanStopWordsList.append("viel")
    germanStopWordsList.append("viele")
    germanStopWordsList.append("wegen")
    germanStopWordsList.append("müssen")
    germanStopWordsList.append("geht")
    germanStopWordsList.append("gibt")
    germanStopWordsList.append("wer")
    germanStopWordsList.append("wie")
    germanStopWordsList.append("was")
    germanStopWordsList.append("macht")
    germanStopWordsList.append("machen")
    germanStopWordsList.append("machte")
    germanStopWordsList.append("kommen")
    germanStopWordsList.append("kommt")
    germanStopWordsList.append("glaube")
    germanStopWordsList.append("glaubst")
    germanStopWordsList.append("tun")
    germanStopWordsList.append("wäre")
    germanStopWordsList.append("sagte")
    germanStopWordsList.append("sagten")
    germanStopWordsList.append("hat")
    germanStopWordsList.append("hast")
    germanStopWordsList.append("haben")
    germanStopWordsList.append("habt")
    germanStopWordsList.append("statt")
    germanStopWordsList.append("genau")
    germanStopWordsList.append("sagen")
    germanStopWordsList.append("sagte")
    germanStopWordsList.append("sagten")
    germanStopWordsList.append("bitte")
    germanStopWordsList.append("bitten")
    germanStopWordsList.append("danke")
    germanStopWordsList.append("dank")
    germanStopWordsList.append("sollen")
    germanStopWordsList.append("soll")
    germanStopWordsList.append("sollte")
    germanStopWordsList.append("sehen")
    germanStopWordsList.append("seht")
    germanStopWordsList.append("zeigen")
    germanStopWordsList.append("zeigt")
    germanStopWordsList.append("sei")
    germanStopWordsList.append("sein")
    germanStopWordsList.append("seid")
    germanStopWordsList.append("seit")
    germanStopWordsList.append("laut")
    germanStopWordsList.append("lauten")
    germanStopWordsList.append("sehen")
    germanStopWordsList.append("seht")
    germanStopWordsList.append("haben")
    germanStopWordsList.append("hat")
    germanStopWordsList.append("hätten")
    germanStopWordsList.append("sagte")
    germanStopWordsList.append("sag")
    germanStopWordsList.append("sagt")
    germanStopWordsList.append("ab")
    germanStopWordsList.append("bei")
    germanStopWordsList.append("beim")
    germanStopWordsList.append("denen")
    germanStopWordsList.append("gab")
    germanStopWordsList.append("ab")
    
    for fItem in filterList:
        germanStopWordsList.append(fItem)
    
    print("- Start generate cloud")
    wordcloud = WordCloud(
                background_color="black",
                width=1920,
                height=1080,
                stopwords=germanStopWordsList
            ).generate(textString)
    wordcloud.to_file(dir_var_output + "wordcloud-" + label + ".png")
    
    print()
    print("Top 20 occ:\n" + str(pd.Series(wordcloud.words_).head(20)))
    print()
    
    print("- Start generate figure")
    plt.figure(figsize=(14, 14))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.show()
    
    gloStopStopwatch("Word Cloud")

In [ ]:
# Oliver Janich öffentlich (public_channel)
plotWordCloud(
    "DS-08-10-2020/ChatExport_2020-09-25-janich",
    "pc-janich",
    []
)

In [ ]:
# Eva Herman Offiziell (public_channel)
plotWordCloud(
    "DS-08-10-2020/ChatExport_2020-09-27-evaherman",
    "pc-evaHerman",
    []
)

In [ ]:
# ATTILA HILDMANN OFFICIAL (public_channel)
plotWordCloud(
    "DS-08-10-2020/ChatExport_2020-09-25-hildmann",
    "pc-hildmann",
    ["ATTILAHILDMANN CHAT"]
)

In [ ]:
# Xavier Naidoo (public_channel)
plotWordCloud(
    "DS-08-10-2020/ChatExport_2020-09-25-xavier",
    "pc-xavier",
    ["xavier_naidoo", "Xavier_Naidoo", "politische_bildersprueche", "einmal_hin_alles_drin"]
)

In [ ]:
# Just for test purposes
plotWordCloud(
    "global",
    "global",
    []
)

## Test Stuff

In [ ]:
#G_weighted = nx.DiGraph()

#G_weighted.add_edge('A', 'B', weight=8)
#G_weighted.add_edge('A', 'C', weight=2)
#G_weighted.add_edge('A', 'D', weight=5)
#G_weighted.add_edge('C', 'D', weight=3)
#G_weighted.add_edge('A', 'C', weight=2)
#G_weighted.add_edge('D', 'C', weight=10)
#G_weighted.add_edge('C', 'E', weight=5)

#G_weighted.add_node('A', weight=500)
#G_weighted.add_node('C', weight=300)
#G_weighted.add_node('D', weight=500)
#G_weighted.add_node('B', weight=600)
#G_weighted.add_node('E', weight=500)

In [ ]:
#from ipywidgets import interact
#import ipywidgets as widgets

In [ ]:
#interact(plot_random_graph, n=(2,30), m=(1,10), k=(1,10), p=(0.0, 1.0, 0.001),
#         generator={
#             'lobster': random_lobster,
#             'power law': powerlaw_cluster,
#             'Newman-Watts-Strogatz': newman_watts_strogatz,
#             u'Erdős-Rényi': erdos_renyi,
#         });

### Concordance

In [ ]:
#tbd